In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
import os

# Put your OpenAI API keys here
client = OpenAI(api_key="OPENAI_API_KEY")

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
  ### Fill the function
    messages = [{"role": "user", "content": prompt}]

    # Call the OpenAI API to generate a chat completion using the specified model and messages
    # The temperature is set to 0 for deterministic output
    response = client.chat.completions.create(model=model,
                                              messages=messages,
                                              temperature=0)

    # Return the content of the first message in the response's choices
    return response.choices[0].message.content

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp /content/drive/MyDrive/ITP459/final_project/data/*.csv /content
!cp /content/drive/MyDrive/ITP459/final_project/data/NVDA_Price.csv /content

Mounted at /content/drive


In [ ]:
import pandas as pd
# df_news = pd.read_csv('NVDA_news.csv')
# df_price = pd.read_csv('NVDA_Price.csv')
# df_news.head()

In [ ]:
def create_combined_csv(input_news_csv_files, input_price_csv_file, output_combined_filename):
    df_new = {
        'Date': [],
        'Description': [],
        'Author': [],
        'Sentiment': [],
        'Severity': [],
        'Stock_Price_difference': [],
        'Stock_Price_Percentage_difference': []
    }
    for i in range(len(input_news_csv_files)):
        news_csv_file = input_news_csv_files[i]
        df_news = pd.read_csv(news_csv_file)
        df_price = pd.read_csv(input_price_csv_file)
        df_price['Date'] = pd.to_datetime(df_price['Date'])
        df_news['Date'] = pd.to_datetime(df_news['Date'])
        merged_df = pd.merge(df_price, df_news, on='Date', how='inner')
        for index, row in merged_df.iterrows():
            date = pd.to_datetime(row['Date'], format='%m/%d/%Y')
            print(date)
            next_day = date + pd.Timedelta(days=1)
            price_date = df_price[df_price['Date']==date]
            price_next_day = df_price[df_price['Date']==next_day]
            news_description = row['Description']
            news_author = row['Author']
            prompt = prompt = f"""
            Your task is to give sentiment analysis for this news article regarding Nvidia or another company. Only provide your response in one of Positive, Negative, and Neutral.

            News: '''{news_description}'''
            """
            response_sentiment = get_completion(prompt)

            prompt_severity = f"""
            Your task is to provide a severity score based on the content of this news article and its author.
            A high severity score means the news content and the creidibility of the author have a high impact on Nvidia's stock price.
            A low severity score means the news content and the creidibility of the author will not likely have an impact on Nvidia's stock price.
            Provide your response on a scale of 1 to 5 only. A score of 5 means highest severity, score of 0 means lowest severity.
            Please provide your response with the severity score only and no explanation.

            News: '''{news_description}'''
            Author: '''{news_author}'''
            """
            res_severity = get_completion(prompt_severity)

            print(news_description)
            print(res_severity, response_sentiment)


            if not price_date.empty and not price_next_day.empty:
                df_new['Date'].append(date)
                df_new['Description'].append(news_description)
                df_new['Author'].append(news_author)
                df_new['Sentiment'].append(response_sentiment)
                df_new['Severity'].append(res_severity)

                # Access the first row of the filtered DataFrame
                price_diff = price_next_day.iloc[0]['Close'].astype(float) - price_date.iloc[0]['Open'].astype(float)
                df_new['Stock_Price_difference'].append(price_diff)
                df_new['Stock_Price_Percentage_difference'].append(price_diff / price_date.iloc[0]['Open'].astype(float))
                print(price_diff)

            elif not price_date.empty and price_next_day.empty: # date is a friday, next day is a weekend
                print('Next day is Weekend')
                next_day = date + pd.Timedelta(days=3)
                price_next_day = df_price[df_price['Date']==next_day]
                if price_next_day.empty: # end of dataset, this exceeds maximum dates recorded
                    continue
                df_new['Date'].append(date)
                df_new['Description'].append(news_description)
                df_new['Author'].append(news_author)
                df_new['Sentiment'].append(response_sentiment)
                df_new['Severity'].append(res_severity)
                price_diff = price_next_day.iloc[0]['Close'].astype(float) - price_date.iloc[0]['Open'].astype(float)
                df_new['Stock_Price_difference'].append(price_diff)
                df_new['Stock_Price_Percentage_difference'].append(price_diff / price_date.iloc[0]['Open'].astype(float))

                print(price_diff)
            else:
                print('No data available')

    df_new = pd.DataFrame(df_new)
    df_new.to_csv('NVDA_combined_data.csv', index=False)





In [ ]:
news_csv_files = ['AMZN_news.csv', 'GOOG_news.csv', 'META_news.csv', 'NVDA_news.csv', 'Oracle_news.csv', 'Samsung_news.csv', 'TSLA_news.csv', 'WMT_news.csv']

create_combined_csv(news_csv_files, 'NVDA_Price.csv', 'NVDA_combined_data.csv')

2024-04-30 00:00:00
Kobo’s latest e-reader introduces a color E Ink screen that brings graphic novels, children’s books, and your favorite book covers to life.
2 Positive
Next day is Weekend
2024-04-30 00:00:00
Amazon says it's working on stopping others from "making your AWS bill explode."
3 Neutral
Next day is Weekend
2024-04-30 00:00:00
Amazon's Q1 2024 revenue increased by 13% year over year, thanks to an increase in AI investments and companies looking to AWS to power their services.
2 Positive
Next day is Weekend
2024-04-30 00:00:00
Fallout — the Amazon Prime Video TV series — isn’t just a critical success. Amazon says the series drew 65… Continue reading Fallout pulls record viewer numbers for Amazon TV
The post Fallout pulls record viewer numbers for Amazon TV appeared first on ReadWri…
2 Positive
Next day is Weekend
2024-04-30 00:00:00
Amazon said it delivered over 2 billion items "the same or next day" last quarter.
2 Positive
Next day is Weekend
2024-04-29 00:00:00
If you've